In [1]:
import json
from glob import glob
import fnmatch
from multiprocessing import Pool
from pathlib import Path
import os, itertools
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from pickle import dumps, loads, dump, load

from stexls import *

%cd ~/MathHub

/home/marian/MathHub


In [2]:
%rm /tmp/linker.bin

In [3]:
try:
    with open('/tmp/linker.bin', 'rb') as fd:
        linker = load(fd)
except Exception as e:
    print(e)
    linker = Linker('.', limit=None)
    linker.update(progress=tqdm)
    with open('/tmp/linker.bin', 'w+b') as fd:
        dump(linker, fd)

[Errno 2] No such file or directory: '/tmp/linker.bin'


MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"smglom/smglom/source/alpharenaming.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/talks/source/arxmliv/en/arxmliv.tex:6:1: File not indexed:"BaseMH/physics/source/units/en/siunits-prefixed.tex"
MiKoMH/CompLog/source/trs/en/intro.tex:2:1: File not indexed:"smglom/smglom/source/abstract-reduction-system.tex"
MiKoMH/talks/source/kwarc/en/projects.tex:5:1: File not indexed:"MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
MiKoMH/talks/source/sTeX/en/stexide.tex:2:1: File not indexed:"MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
MiKoMH/talks/source/planetary/en/realizing.tex:8:3: File not indexed:"MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
MiKoMH/CompLog/source/trs/en/confluent.tex:2:1: File not indexed:"smglom/smglom/source/abstract-reduction-system.tex"
MiKoMH/AI/source/ml/en/brains.tex:2:3: File not indexed:"BaseMH/physics/source/uni

MiKoMH/talks/source/mathhub/en/nutshell2.tex:9:1: File not indexed:"MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
Undefined module: "dtl-ago" not defined in "MiKoMH/AI/source/ml/en/dtl-algo.tex"
Undefined module: "decisions-network" not defined in "MiKoMH/AI/source/rational-decisions/en/decision-network.tex"
MiKoMH/talks/source/formalcad/en/flange-ontology.tex:2:1: File not indexed:"KwarcMH/FormalCAD/source/flange/cds/ISOmetricthread.tex"
Undefined module: "OEIS-generating-functions" not defined in "MiKoMH/talks/source/OEIS/en/generating-functions.tex"
Undefined module: "category" not defined in "MiKoMH/KRMT/source/categories/en/category.tex"
MiKoMH/talks/source/eLearning/talks/forcnx.tex:34:1: File not indexed:"MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
Undefined module: "abstract-consistency" not defined in "MiKoMH/CompLog/source/hopl/en/abstract-consistency.tex"
Undefined module: "authentication" not defined in "MiKoMH/SecDep/source/auth/en/ssh-authentication.tex"
MiKoMH/GenCS/

In [4]:
%ls -ahl /tmp/linker.bin

-rw-rw-r-- 1 marian marian 61M Apr  5 16:36 /tmp/linker.bin


In [5]:
path = Path('MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex')

In [6]:
for object in linker.objects[path]:
    print(linker.links[object].format())
    print()
    print('-'*80)
    print()

Contains files:
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex

Depends on:
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:9:3:public pairs from "MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex"
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:9:25:public sets from "MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex"
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:20:3:public setoid from "MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex"

Symbols:
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:3:19:public sets/MODULE
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:4:3:public sets.member/SYMBOL
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:5:3:public sets.mmember/SYMBOL
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:1:19:public pairs/MODULE
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:1:25:public pairs.pair/SYMBOL
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:8:19:public setoid/MODULE
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:10:3:public setoid.sset/SYMBOL
	MiKoMH

In [20]:
linker.view_import_graph(path)

In [19]:
for object in linker.objects[path]:
    print(linker.links[object].format())
    print()
    print('-'*80)
    print()

Contains files:
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex

Depends on:
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:9:3:public pairs from "MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex"
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:9:25:public sets from "MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex"
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:20:3:public setoid from "MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex"

Symbols:
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:3:19:public sets/MODULE
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:4:3:public sets.member/SYMBOL
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:5:3:public sets.mmember/SYMBOL
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:1:19:public pairs/MODULE
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:1:25:public pairs.pair/SYMBOL
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:8:19:public setoid/MODULE
	MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex:10:3:public setoid.sset/SYMBOL
	MiKoMH